In [1]:
import numpy as np

dataset =['hdb-resale-flat-prices/resale-flat-prices-based-on-approval-date-1990-1999.csv', 
          'hdb-resale-flat-prices/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv', 
          'hdb-resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv', 
          'hdb-resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv']

# read in all dataset & combined into one (named: consolidated_db)
for num, fname in enumerate(dataset):
    print('Reading from {}'.format(fname))
    data= np.genfromtxt(fname,skip_header=1,dtype=[('year_month','U10'),
                                                   ('town','U30'),
                                                   ('flat_type','U10'),
                                                   ('block','U6'),
                                                   ('street_name','U40'),
                                                   ('storey_range','U20'),
                                                   ('floor_area','i4'),
                                                   ('flat_model','U15'),
                                                   ('lease_commence_date','i8'),
                                                   ('resale_price','i8')],delimiter=",")
    if num == 0: consolidated_db = data
    else: consolidated_db = np.concatenate((consolidated_db,data))
print('Total Number of Records {}'.format(len(consolidated_db)))

#  Extract Unique Keys from Database
Town_List = np.unique(consolidated_db['town'])
Flat_Type_List = np.unique(consolidated_db['flat_type'])
Flat_Type_List = Flat_Type_List[:][0:-2] #get rid of Multi_Gen Type of Flats
Storey_Range_List = np.unique(consolidated_db['storey_range'])
Year_List = np.arange(1990,2019,1).astype('str')

# Functions to extract Boolean Index of Consolidated Database...                

def by_year(year):
    year_mth = np.char.split(consolidated_db['year_month'],sep='-')
    bool_idx = np.empty(len(consolidated_db)).astype('bool')                     
    for i in np.arange(len(consolidated_db)): 
        bool_idx[i] = ((year_mth[i][0][0:4] == year))
    return bool_idx
def by_town(town):
    bool_idx = (consolidated_db['town']==town)                 
    return bool_idx
def by_flat_type(flat_type):
    bool_idx = (consolidated_db['flat_type']==flat_type)
    return bool_idx
def by_storey_range(storey_range):
    bool_idx = (consolidated_db['storey_range']==storey_range)
    return bool_idx
def by_year_mth(year_mth):
    bool_idx = (consolidated_db['year_month']==year_mth)
    return bool_idx


Reading from hdb-resale-flat-prices/resale-flat-prices-based-on-approval-date-1990-1999.csv
Reading from hdb-resale-flat-prices/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
Reading from hdb-resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv
Reading from hdb-resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv
Total Number of Records 785798


In [2]:
import csv
import requests
import numpy as np
from utils.svy21 import SVY21

attrib_list = {'by_year':'2017', 'by_year_mth':'Null', 
               'by_town':'BISHAN', 'by_flat_type':'5 ROOM', 
               'by_storey_range': 'Null'}

first_item = 'True'
for num, key in enumerate(attrib_list):
    string = key+"('"+attrib_list[key]+"')"
    if (attrib_list[key] != 'Null'):
        if(first_item == 'True'):
            bool_idx = eval(string)
            first_item = 'False'
        else:
            bool_idx = np.logical_and(bool_idx,eval(string))


# bool_idx = np.logical_and(by_year('2018'),by_town('BISHAN'))
# #bool_idx = np.logical_and(by_year_mth('2018-04'),by_town('TOA PAYOH'))
# bool_idx = np.logical_and(bool_idx,by_flat_type('5 ROOM'))

db_extracted = consolidated_db[bool_idx]

url1 = 'https://developers.onemap.sg/commonapi/search?searchVal='
url3 = '&returnGeom=Y&getAddrDetails=N&pageNum=1'

with open('data/Property_Info.csv', mode='w', newline='') as csv_file:
    fieldnames = ['Address', 'Latitude', 'Longitude','Storey_Range','Lease_Commence','Resale_Price']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writerow({'Address': 'Address', 
                     'Latitude': 'Latitude', 
                     'Longitude': 'Longitude',
                     'Storey_Range' : 'Storey_Range', 
                     'Lease_Commence': 'Lease_Commence',
                     'Resale_Price' : 'Resale_Price'})
    for record in db_extracted:
        address = record['block'] + ' ' + record['street_name']
        URL = url1 + address + url3 # sending get request and saving the response as response object 
        req = requests.get(url = URL) # extracting data in json format 
        data = req.json()
        for result in data['results']:
            lat = float(result['LATITUDE'])
            lon = float(result['LONGITUDE'])
#        print(address, lat, lon)
        writer.writerow({'Address': address, 
                         'Latitude': lat, 
                         'Longitude': lon,
                         'Storey_Range' : record['storey_range'], 
                         'Lease_Commence': record['lease_commence_date'],
                         'Resale_Price' : record['resale_price']})
    print('Number of records extracted: ', len(db_extracted))

Number of records extracted:  114


In [4]:
import shapefile as shp
import numpy as np
import folium
import webbrowser, os
import math

from utils.svy21 import SVY21
svy = SVY21()

# Get List of MRT Station
MRT_Stn = np.genfromtxt('data\MRT_Stations.csv',skip_header=1, 
                        dtype=[('Station_Name','U30'),
                               ('Latitude','float64'),
                               ('Longitude','float64')],delimiter=",")

# Function to get distance between 2 points in Lat-Long Format
def latlon_dist(origin, destination):
#   example print(latlon_dist((lat1,lon1),(lat2,lon2)))    
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d  # distance in km

# Function to get MRT_Stn 
def nearest_MRT(lat,lon):
    dist = []
    for num, station in enumerate(MRT_Stn):
        dist.append(latlon_dist((lat,lon),(station['Latitude'],station['Longitude'])))  
    idx = dist.index(min(dist))
    return (MRT_Stn['Station_Name'][idx]+ ", Dist =" + 
            str(round(min(dist),2))+"km") #return the MRT Name and dist


# initiate the Leaflet Map
sg_map = folium.Map(location=[1.38, 103.85], zoom_start=12, 
                    no_wrap = True, max_bounds = True)

#  Get Town Boundaries from shapefile
fname_in = "data\Singapore_Town_Boundaries.shp"
sf = shp.Reader(fname_in)
fname_out = "Resale_Map_Bishan_5_Room_2017.html"
records = sf.shapeRecords()

for record in records:
    for i in range(len(record.shape.parts)):
        i_start = record.shape.parts[i]
# check if polygon or multi-polygon in shapefile
        if i==len(record.shape.parts)-1:       
            i_end = len(record.shape.points)
        else:
            i_end = record.shape.parts[i+1]
#
        x = [i[0] for i in record.shape.points[i_start:i_end]]   # extract x,y in
        y = [i[1] for i in record.shape.points[i_start:i_end]]   # SVY coordinates
#  convert to Lat-Long List Comprehension
        LatLon = [svy.computeLatLon(y[num],x[num]) for num in np.arange(len(x))]  
#  Add the Polygon to the Leaflet Map
        folium.PolyLine(LatLon).add_to(sg_map)

################################################

Property_Info = np.genfromtxt('data\Property_Info.csv',skip_header=1, 
                        dtype=[('Address','U30'),
                               ('Latitude','float'),
                               ('Longitude','float'),
                               ('Storey_Range','U20'),
                               ('Lease_Commence','i4'),
                               ('Resale_Price', 'i8')],delimiter=",")

List_MRT_Stn = []

for property in Property_Info:    
    popup1 = (property['Address'] + '<br>' + 
             'Price: '+ str(property['Resale_Price']) + '<br>' +
             'Lease Start: ' + str(property['Lease_Commence']))   
    popup2 = ('Nearest MRT Station: ' + 
              nearest_MRT(property['Latitude'],property['Longitude']))
    List_MRT_Stn.append(nearest_MRT(property['Latitude'],property['Longitude']).split(',',1)[0])
    folium.Marker([property['Latitude'], property['Longitude']],
                  tooltip = popup1, popup = popup2).add_to(sg_map)

List_MRT_Stn = set(List_MRT_Stn)  # convert List to Set to filter out unique stations

#print(List_MRT_Stn)

for station in List_MRT_Stn:
    idx = np.where(MRT_Stn['Station_Name'] == station)
    station_name = MRT_Stn['Station_Name'][idx]
    lat = MRT_Stn['Latitude'][idx]
    lon = MRT_Stn['Longitude'][idx]
    folium.RegularPolygonMarker(
        [lat, lon],
        fill_color='#ff0000',
        fill_opacity = 0.8,
        number_of_sides=36,
        radius=15
        ).add_to(sg_map)

sg_map.save(fname_out, close=True)
webbrowser.open('file://' +os.path.realpath(fname_out))


True